In [1]:
import pandas as pd
import numpy as np
import numba

In [3]:
#lazy compilation
from numba import jit

@jit
def f(x, y):
    return x + y

In [11]:
#eager compilation
from numba import int32, float32

@jit(int32(int32, int32))
def f(x, y):
    return x + y

f(2.5, 3)

5

In [43]:
#working with arrays
@jit(int32[:](int32))
def t(x):
    return np.arange(x)


def p(x):
    return np.arange(x)

%timeit _ = t(1000000)
#%timeit _ = p(1000000)

1000 loops, best of 3: 189 µs per loop


In [ ]:
#compilation options
#give the tip-top performance but can raise errors
@jit(int32[:](int32), nopython=True)
def t(x):
    return np.arange(x)


#when code is only working with native types and variables 
#aka no python objects, we can release GIL
#code with nogil=True and nopython=True will run concurrently
@jit(nogil=True)



> While the jit() decorator is useful for many situations, sometimes you want to write a function that has different implementations depending on its input types. The generated_jit() decorator allows the user to control the selection of a specialization at compile-time, while fulling retaining runtime execution speed of a JIT function.


> Suppose you want to write a function which returns whether a given value is a “missing” value according to certain conventions. For the sake of the example, let’s adopt the following definition:

> for floating-point arguments, a missing value is a NaN
for Numpy datetime64 and timedelta64 arguments, a missing value is a NaT
other types don’t have the concept of a missing value.

In [44]:
import numpy as np

from numba import generated_jit, types

@generated_jit(nopython=True)
def is_missing(x):
    """
    Return True if the value is missing, False otherwise.
    """
    if isinstance(x, types.Float):
        return lambda x: np.isnan(x)
    elif isinstance(x, (types.NPDatetime, types.NPTimedelta)):
        # The corresponding Not-a-Time value
        missing = x('NaT')
        return lambda x: x == missing
    else:
        return lambda x: False

There are several things to note here:

1. The decorated function is called with the Numba types of the arguments, not their values.
2. The decorated function doesn’t actually compute a result, it returns a callable implementing the actual definition of the function for the given types.
3. It is possible to pre-compute some data at compile-time (the missing variable above) to have them reused inside the compiled implementation.
4. The function definitions use the same names for arguments as in the decorated function, this is required to ensure passing arguments by name works as expected.


### Creating Numpy universal functions

Numba’s vectorize allows Python functions taking scalar input arguments to be used as NumPy ufuncs.

In [86]:
from numba import vectorize, float64, int64


@vectorize([int32(int32, int32),
            int64(int64, int64),
            float32(float32, float32),
            float64(float64, float64)], target="cpu")
def f(x, y):
    return x + y

a = np.arange(10)
%time f(a, a)

#good thing is f also gets reduction, accumulation or broadcasting
print f.accumulate(a)
print f.reduce(a)

CPU times: user 11 µs, sys: 1e+03 ns, total: 12 µs
Wall time: 13.8 µs
[ 0  1  3  6 10 15 21 28 36 45]
45


> If you pass several signatures, beware that you have to pass most specific signatures before least specific ones (e.g., single-precision floats before double-precision floats), otherwise type-based dispatching will not work as expected

> target can be "cpu", "parallel" or "cuda", targeting data less than 1KB, less than 1MB and more than 1 MB
for example with array 1000000 * 100, should be using parallel

In [33]:
import json
import urllib3.request
import pandas

url = urllib3.request.urlopen('https://www.cryptocompare.com/api/data/coinlist/')
json_obj = json.load(url)
print(json.dumps(json_obj, sort_keys=True, indent=4))


AttributeError: 'module' object has no attribute 'urlopen'

In [36]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.25)
t = np.arange(0.0, 1.0, 0.001)
a0 = 5
f0 = 3
s = a0*np.sin(2*np.pi*f0*t)
l, = plt.plot(t, s, lw=2, color='red')
plt.axis([0, 1, -10, 10])

axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03])
axamp = plt.axes([0.25, 0.15, 0.65, 0.03])

sfreq = Slider(axfreq, 'Freq', 0.1, 30.0, valinit=f0)
samp = Slider(axamp, 'Amp', 0.1, 10.0, valinit=a0)


def update(val):
    amp = samp.val
    freq = sfreq.val
    l.set_ydata(amp*np.sin(2*np.pi*freq*t))
    fig.canvas.draw_idle()
sfreq.on_changed(update)
samp.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    sfreq.reset()
    samp.reset()
button.on_clicked(reset)

rax = plt.axes([0.025, 0.5, 0.15, 0.15])
radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)


def colorfunc(label):
    l.set_color(label)
    fig.canvas.draw_idle()
radio.on_clicked(colorfunc)

plt.show()

In [20]:
result

,Number_of_unique_authors,Time
0,1,2015-05-13 01:12:12
1,2,2015-05-12 02:12:12
2,3,2015-05-12 04:12:12
3,3,2015-05-13 03:12:12
